In [ ]:
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os
import numpy as np

In [2]:
SEED = 42
log_path = os.path.join('Training', 'Logs')
model_path = os.path.join('Training', 'Saved Models')

In [3]:
env = make_atari_env("ALE/Breakout-v5", n_envs=4, seed=SEED)
env = VecFrameStack(env, n_stack=4)

In [4]:
# episodes = 5
# for episode in range(episodes):
#     state = env.reset()
#     done = np.array([False])
#
#     while not done.all():
#         env.render()
#         action = env.action_space.sample()
#         n_state, reward, done, info = env.step([action]*os.cpu_count())
#         print(done)

In [5]:
# env.close()

In [6]:
model = PPO("CnnPolicy", env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [7]:
if os.path.exists(os.path.join(model_path, 'PPO_100k.zip')):
    model = PPO.load(os.path.join(model_path, 'PPO_100k.zip'), env=env)
else:
    model.learn(100000)
    model.save(os.path.join(model_path, 'PPO_100k'))

Wrapping the env in a VecTransposeImage.


In [8]:
evaluate_policy(model=model, env=env, render=True, n_eval_episodes=10)

(20.4, 3.7735924528226414)

In [9]:
env.close()

In [14]:
if os.path.exists(os.path.join(model_path, 'PPO_1M.zip')):
    model = PPO.load(os.path.join(model_path, 'PPO_1M.zip'), env=env)
else:
    model.learn(1000000)
    model.save(os.path.join(model_path, 'PPO_1M'))

Wrapping the env in a VecTransposeImage.


In [15]:
evaluate_policy(model=model, env=env, render=True, n_eval_episodes=10)

(14.5, 6.606814663663572)

стоило сразу коллбэк повесить, а то произошли сильные просадки про скору и надо вытащить из этой ямы

In [16]:
env.reset()
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=20, verbose=1)
eval_callback = EvalCallback(
    env,
    callback_on_new_best=stop_callback,
    eval_freq=10000,
    best_model_save_path=model_path,
    verbose=1
)

In [17]:
model.learn(100000, callback=eval_callback)

Logging to Training\Logs\PPO_5


C:\Python310\lib\site-packages\stable_baselines3\common\callbacks.py:403: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_transpose.VecTransposeImage object at 0x000001A759F36A10> != <stable_baselines3.common.vec_env.vec_frame_stack.VecFrameStack object at 0x000001A5B2F1AFB0>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 700      |
|    ep_rew_mean     | 17       |
| time/              |          |
|    fps             | 219      |
|    iterations      | 1        |
|    time_elapsed    | 37       |
|    total_timesteps | 8192     |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 665       |
|    ep_rew_mean          | 15.7      |
| time/                   |           |
|    fps                  | 181       |
|    iterations           | 2         |
|    time_elapsed         | 90        |
|    total_timesteps      | 16384     |
| train/                  |           |
|    approx_kl            | 0.4739682 |
|    clip_fraction        | 0.439     |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.457    |
|    explained_variance   | 0.579     |
|    learning_rate        | 0.0003    |
|    loss           

In [19]:
env.reset()
best = PPO.load(os.path.join(model_path, 'best_model.zip'), env=env)
evaluate_policy(model=best, env=env, render=True, n_eval_episodes=50)

Wrapping the env in a VecTransposeImage.


(17.92, 6.045957327007858)

Во время тренировки были пики со скорами ~20, можно было там сохранить. Так же можно поподбирать гиперпараметры, но допустим что это хоть какой-то результат для первой сдачи, поэтому пока оставлю так.